In [1]:
from codebase.classes import Particles
from codebase.classes_data import Data
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from codebase.ibis import essl, exp_and_normalise, run_stan_model
from run_ibis import run_ibis

from tqdm import tqdm
from codebase.plot import plot_density, get_post_df, plot_correlations
import altair as alt
alt.data_transformers.disable_max_rows()

%load_ext autoreload
%autoreload 2

## 2 factor Sim AZ

In [2]:
existing_directory = None
task_handle = 'az_comp'
gen_model = 0

if existing_directory is None:
    log_dir = make_folder(task_handle)  
    print("\n\nCreating new directory: %s" % log_dir)

else:
    log_dir = existing_directory
    log_dir = path_backslash(log_dir)
    print("\n\nReading from existing directory: %s" % log_dir)




Creating new directory: ./log/20201228_164655_az_comp/


## Create Data

In [3]:
# generate data
exp_data = Data(
    name = task_handle, 
    model_num = 4, 
    size = 100,
    random_seed = 6
    )
    
exp_data.generate()

## Run HMC

In [4]:
sm = load_obj('sm', 'log/compiled_models/model3/')

fit_run = run_stan_model(
    data = exp_data.get_stan_data(),
    compiled_model = sm,
    num_samples = 1000, 
    num_warmup = 1000,
    num_chains = 1,
    adapt_engaged=True
    )

In [5]:
param_names = ['beta', 'alpha', 'Marg_cov']

ps = fit_run.extract(
        permuted=False, pars=param_names)

save_obj(ps, 'ps_hmc', log_dir)

## Run IBIS

In [6]:
gen_model = 0
model_num = 3

ibis = run_ibis(
    exp_data,
    model_num,
    500,
    gen_model,
    log_dir
    )

for name in ['alpha', 'Marg_cov']:
    samples = np.squeeze(ibis['particles'].particles[name])
    w = exp_and_normalise(ibis['particles'].weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    # print('\nRead Data')
    # print(np.round(exp_data.raw_data[name],2))

100%|██████████| 100/100 [08:58<00:00,  5.39s/it]




Marginal Likelihood 0.09790


Estimate
[-0.02 -0.02  0.06 -0.04 -0.05  0.  ]


Estimate
[[0.84 0.3  0.33 0.12 0.09 0.08]
 [0.3  0.75 0.37 0.07 0.07 0.04]
 [0.33 0.37 0.88 0.03 0.02 0.08]
 [0.12 0.07 0.03 0.95 0.41 0.3 ]
 [0.09 0.07 0.02 0.41 1.01 0.31]
 [0.08 0.04 0.08 0.3  0.31 0.84]]


## Plot Both results

In [7]:
# have to resample particles to get rid of weights
particles = ibis['particles']
particles.resample_particles()

In [8]:
param = 'beta'
df = get_post_df(ps[param]) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))&\
plot_correlations(ibis['jitter_corrs'][param], height = 150)

INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


alt.VConcatChart(...)

In [9]:
param = 'alpha'
df = get_post_df(ps[param]) 
df['source'] = 'hmc'
df2 = get_post_df(particles.particles[param])
df2['source'] = 'ibis'

plot_density(pd.concat([df,df2]))&\
plot_correlations(ibis['jitter_corrs'][param], height = 150)

alt.VConcatChart(...)